In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
#Dataset used: https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000

path='/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv'

In [ ]:
df=pd.read_csv(path)
df.tail()

In [ ]:
fractions=np.array([0.8,0.2])

df=df.sample(frac=1)

train_set, test_set = np.array_split(df, (fractions[:-1].cumsum() * len(df)).astype(int))

In [ ]:
print(len(train_set))
print(len(test_set))
df.label.unique()

In [ ]:
# reference: https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000/discussion/183083
classes={
    0:('akiec', 'actinic keratoses and intraepithelial carcinomae'),
         
    1:('bcc' , 'basal cell carcinoma'),
         
    2:('bkl', 'benign keratosis-like lesions'),
         
    3:('df', 'dermatofibroma'),
         
    4:('nv', ' melanocytic nevi'),
         
    5:('vasc', ' pyogenic granulomas and hemorrhage'),
         
    6:('mel', 'melanoma'),
}

In [ ]:
y_train=train_set['label']

x_train=train_set.drop(columns=['label'])

y_test=test_set['label']

x_test=test_set.drop(columns=['label'])

columns=list(x_train)

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
# pip install --upgrade scikit-learn imbalanced-learn


In [ ]:
# pip uninstall scikit-learn imbalanced-learn -y


In [ ]:
# # pip install scikit-learn==1.3.0 imbalanced-learn==0.10.1
# !pip install scikit-learn==1.6.1 imbalanced-learn==0.13.0


In [ ]:
from imblearn.over_sampling import RandomOverSampler 

oversample = RandomOverSampler()

x_train,y_train  = oversample.fit_resample(x_train,y_train)

In [ ]:
import numpy as np
import tensorflow as tf

# Step 1: Convert list to array if it's a list of images
# x_train = np.array(x_train)
x_train = x_train.values.reshape(-1, 28, 28, 3)
# Step 2: Ensure type is float32 and normalized
x_train = x_train.astype("float32") / 255.0

# Step 3: Ensure it has 4D shape: (N, H, W, C)
print("Before resize, x_train shape:", x_train.shape)

# Step 4: Resize all images to (75, 100)
x_train = tf.image.resize(x_train, (75, 100)).numpy()

# Final check
print("After resize, x_train shape:", x_train.shape)  # should be (N, 75, 100, 3)


In [ ]:
import matplotlib.pyplot as plt
import random
import tensorflow as tf
num=random.randint(0,8000)
# x_train=np.array(x_train, dtype=np.uint8).reshape(-1,28,28,3)
# Normalize and resize x_train properly
x_train = np.array(x_train, dtype=np.float32) / 255.0
x_train = tf.image.resize(x_train, (75, 100)).numpy()

# One-hot encode y_train (if needed)
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=7)

print("x_train shape:", x_train.shape)  # Should be (N, 75, 100, 3)
print("y_train shape:", y_train.shape)  # Should be (N, 7)
print("x_train dtype:", x_train.dtype)  # Should be float32

# plt.imshow(x_train[num].reshape(28,28,3))
plt.title("Random image from training data")
plt.show()
num=random.randint(0,8000)
# plt.imshow(x_train[num].reshape(28,28,3))
plt.title("Random image from training data")
plt.show()

num=random.randint(0,8000)
# plt.imshow(x_train[num].reshape(28,28,3))
plt.title("Random image from training data")
plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
import tensorflow as tf

In [ ]:
# #https://keras.io/api/models/sequential/
# #https://keras.io/api/layers/core_layers/dense/
# #https://keras.io/api/layers/merging_layers/add/
# #https://keras.io/api/layers/convolution_layers/convolution2d
# #https://keras.io/api/layers/convolution_layers/convolution2d
# #https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization


%time 

model = Sequential()

model.add(Conv2D(16, 
                 kernel_size = (3,3), 
                 input_shape = (75, 100, 3), 
                 activation = 'relu', 
                 padding = 'same'))

model.add(MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(32, 
                 kernel_size = (3,3), 
                 activation = 'relu'))

model.add(Conv2D(64, 
                 kernel_size = (3,3), 
                 activation = 'relu'))

model.add(MaxPool2D(pool_size = (2,2)))

model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(128, 
                 kernel_size = (3,3), 
                 activation = 'relu'))

model.add(Conv2D(256, 
                 kernel_size = (3,3), 
                 activation = 'relu'))

model.add(Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(256,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(128,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(64,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(32,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(7,activation='softmax'))

model.summary()



In [ ]:
#reference: https://www.kaggle.com/dhruv1234/ham10000-skin-disease-classification

callback = tf.keras.callbacks.ModelCheckpoint(filepath='new_model.h5',
                                              monitor='val_acc', 
                                              mode='max',
                                              verbose=1,
                                              save_best_only=True)

In [ ]:
%time 

optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss = 'categorical_crossentropy',
              optimizer =optimizer,
              metrics = ['accuracy'])

In [ ]:
print(y_train.shape)
print(type(y_train))


In [ ]:
from datetime import datetime

start_time = datetime.now()

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size = 128,
                    epochs = 50,
                    shuffle=True,
                    callbacks=[callback])

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

In [ ]:
#plot of accuracy vs epoch

plt.plot(history.history['accuracy'])

plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')

plt.ylabel('accuracy')

plt.xlabel('epoch')

plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
#plot of loss vs epoch

plt.plot(history.history['loss'])

plt.plot(history.history['val_loss'])

plt.title('model loss')

plt.ylabel('loss')

plt.xlabel('epoch')

plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:

model.save("/kaggle/working/final_model112.h5")  # or .keras
